In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder

In [2]:
ccfd = pd.read_csv(r'C:\Users\Vijay\Downloads\financial_anomaly_data.csv')

In [3]:
ccfd.head()

,Timestamp,TransactionID,AccountID,Amount,Merchant,TransactionType,Location
0,01-01-2023 08:00,TXN1127,ACC4,95071.92,MerchantH,Purchase,Tokyo
1,01-01-2023 08:01,TXN1639,ACC10,15607.89,MerchantH,Purchase,London
2,01-01-2023 08:02,TXN872,ACC8,65092.34,MerchantE,Withdrawal,London
3,01-01-2023 08:03,TXN1438,ACC6,87.87,MerchantE,Purchase,London
4,01-01-2023 08:04,TXN1338,ACC6,716.56,MerchantI,Purchase,Los Angeles


In [4]:
ccfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217441 entries, 0 to 217440
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Timestamp        216960 non-null  object 
 1   TransactionID    216960 non-null  object 
 2   AccountID        216960 non-null  object 
 3   Amount           216960 non-null  float64
 4   Merchant         216960 non-null  object 
 5   TransactionType  216960 non-null  object 
 6   Location         216960 non-null  object 
dtypes: float64(1), object(6)
memory usage: 11.6+ MB


In [5]:
ccfd[ccfd.isnull().any(axis=1)]

,Timestamp,TransactionID,AccountID,Amount,Merchant,TransactionType,Location
216960,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216961,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216962,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216963,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216964,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
217436,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217437,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217438,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217439,NaN,NaN,NaN,NaN,NaN,NaN,NaN


481 rows are completely null. Hence dropping them

In [6]:
ccfd.dropna(inplace=True)

In [7]:
# Label Encoding for categorical columns
label_encoder = LabelEncoder()
encoder_dict={}
categorical_columns = ['TransactionType', 'Merchant', 'Location']
for col in categorical_columns:
    ccfd[col] = label_encoder.fit_transform(ccfd[col])
    encoder_dict[col]=label_encoder

In [8]:
X = ccfd.drop(['Timestamp', 'TransactionID', 'AccountID'], axis=1) 

In [9]:
# Train the Isolation Forest model
isolation_forest = IsolationForest(n_estimators=100, contamination=0.01)  # Adjust parameters as needed
isolation_forest.fit(X)

IsolationForest(contamination=0.01)

In [10]:
# Predict anomalies
predictions = isolation_forest.predict(X)
ccfd['Anomaly'] = predictions 

In [11]:
# View detected anomalies
detected_anomalies = ccfd[ccfd['Anomaly'] == -1]  # Anomaly label is -1 for Isolation Forest

In [12]:
# Print or inspect the detected anomalies
detected_anomalies

,Timestamp,TransactionID,AccountID,Amount,Merchant,TransactionType,Location,Anomaly
78,01-01-2023 09:18,TXN453,ACC12,1725.94,9,0,0,-1
291,01-01-2023 12:51,TXN349,ACC12,3262.29,9,1,0,-1
350,01-01-2023 13:50,TXN454,ACC12,2573.95,9,0,0,-1
559,01-01-2023 17:19,TXN1217,ACC15,15454.43,9,2,0,-1
590,01-01-2023 17:50,TXN1512,ACC7,241.73,2,0,0,-1
...,...,...,...,...,...,...,...,...
216753,31-05-2023 20:33,TXN545,ACC11,85291.07,9,2,0,-1
216787,31-05-2023 21:07,TXN72,ACC15,96836.33,0,2,0,-1
216803,31-05-2023 21:23,TXN523,ACC14,97458.45,0,0,0,-1
216904,31-05-2023 23:04,TXN1075,ACC9,98376.93,9,0,4,-1


In [13]:
detected_anomalies['Merchant'].value_counts()

9    1205
0     788
1      99
8      42
2      29
Name: Merchant, dtype: int64

In [18]:
decoded_predictions = []

for idx, column in enumerate(categorical_columns):
    label_encoder = encoder_dict[column]
    labels = label_encoder.inverse_transform(detected_anomalies[:, idx])
    decoded_predictions.append(labels)

InvalidIndexError: (slice(None, None, None), 0)